In [ ]:
import pandas as pd
import re, subprocess, pyodbc , os
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
sw=stopwords.words('english')
ps=PorterStemmer()
import json 

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
	SELECT ID, QGROUP, QPK, MAX_Q_PK, QOFFSET, MAX_QG_PK, QGOFFSET, MAX_FK_Form,QTEXT, IdText
	FROM vw_MetricsCompositeKeys WHERE PK_Form = '2020-A-SAOP'   
""" 
df = pd.read_sql(query,con=conn)     
conn.close()

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
    SELECT  *FROM vwPickLists
    WHERE PK_PickListType IN (100,101,102,103,104,105,106,113,114,164,165,166)   
""" 
df_pt = pd.read_sql(query,con=conn) 
conn.close()
df_pt.head(1)

In [ ]:
d = {str(k) : str(k+650) for k in df.QGROUP.unique()  } 
QGOFFSET = 0
QPK = 24500 
for i,r in df.iterrows():      
    if QGOFFSET != r.QGOFFSET:
        QGOFFSET = r.QGOFFSET
        QPK += 24
    d[str(r.QPK)]= str(QPK)
    QPK+=1  

In [ ]:
with open("C:\_som\_src\_compile\saop\keyval.json", "w") as f: 
    json.dump(d, f, indent=4)

In [ ]:
d_re = {}
d_re['PickListType'] = 'PK_PickListType="(\d{1,4})"' 
directory = r'C:\_som\_src\_compile\saop\src'
lst = [] 
for entry in os.scandir(directory):
    if "aspx" in entry.path:
        with open(entry.path, encoding='utf8') as f:
            c = ''.join(f.readlines())  
            d = {}
            d['path']=entry.path
            d['fn']=re.search('(\d{4}_\w{1}_\w{1,8}_\d{1,2})', entry.path).group(1)
            d['ext']= re.search('\.(.*)', entry.path).group(1)  
            for k, v in d_re.items(): 
                d[k]=re.findall(v,c)
            d['content']=c
            lst.append(d) 
dff = pd.DataFrame(lst)        
dff


In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """\
SELECT PK_PickList, PK_PickListType, DisplayValue, CodeValue, SortPos, Description, UsageField FROM vwPickLists
WHERE PK_PickListType IN (100,101,102,103,104,105,106,113,114,164,165,166)
"""  
dfpicks = pd.read_sql(query,con=conn) 
UsageField=dfpicks.iloc[0]['UsageField']
s=f"\n"  
for i,r in dfpicks.iterrows():
    UsageField = r.UsageField
    s = s + f"\tDECLARE @PICK_{UsageField}_{r.CodeValue} INT = {str(r.PK_PickList)}\n" 
print(s)
conn.close()

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """\
SELECT PK_PickList, PK_PickListType, DisplayValue, CodeValue, SortPos, Description, UsageField FROM vwPickLists
WHERE PK_PickListType IN (100,101,102,103,104,105,106,113,114,164,165,166)
""" 
 
dfpicks = pd.read_sql(query,con=conn) 
UsageField=dfpicks.iloc[0]['UsageField']
s=f"\n"
s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[0]['Description']}-{dfpicks.iloc[0]['PK_PickListType']}\n"
for i,r in dfpicks.iterrows():
    if UsageField != dfpicks.iloc[i]['UsageField']:
        UsageField=dfpicks.iloc[i]['UsageField']
        s = s + f"End Enum\n"
        s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[i]['Description']}-{dfpicks.iloc[i]['PK_PickListType']}\n"
    s = s + f"\t{r.CodeValue} = {str(r.PK_PickList)} \t'{str(r.DisplayValue)[:25]}\n" 
s=s+"End Enum\n"
print(s)
conn.close()

In [ ]:
t = """
Has the as Languages technologies platforms is your web don't we have web driver 
"""
def encode(t):
    t = re.sub('[^A-Za-z0-9]', ' ', t ) 
    t = ' '.join([w for w in t.split(' ') if w not in sw])
    #t = ' '.join([ps.stem(w) for w in t.split(' ')])
    t = t.upper()
    t = re.sub('[AEIOU\s]', '', t)  
    return t
encode(t)

In [ ]:
dfid = df
dfid.replace("", np.NaN, inplace=True) 
dfid.dropna(subset = ["IdText"], inplace=True)
dfid['QHASH'] = dfid.QTEXT.apply(encode)  